In [1]:
import jieba
import json
import re
import os
import re
import chardet

In [2]:
# 读取数据集，预处理。
# 分句子
# 句子分词

# 文件批处理
def FileReader(tag):
    # print(os.getcwd())
    path = r'../ex_5/data/htl_del_4000/' + tag 
    os.listdir(path)
    FileList = []
    for iter in os.listdir(path):
        if os.path.splitext(iter)[1] == '.txt':
            FileList.append(iter)
    return FileList


def _PreProcess(article):
    article = re.sub('\\s','',article)
    article = re.sub('，','',article)
    article = re.sub('；','',article)
    article = re.sub('[a-z,A-Z]*','',article)
    sentence = article.split('。|？|！')
    row_list =[]
    for iter in sentence:
        row_list.append(jieba.lcut(iter,cut_all = False)) 
    return row_list

def PreProcess(path,id_f):
    text = ''
    # print(path)
    with open(path,'r',encoding = 'gbk',errors = 'ignore') as txt:
        text = txt.readline()
    # print(text[:20])
    sentences = _PreProcess(text)
    # print(type(sentences))
    dic = {}
    id_c = 0
    for iter in sentences:
        dic[id_c] = iter
        id_c += 1
    return dic



def File_Dic(FileList,tag):
    path = r'../ex_5/data/htl_del_4000/' + tag + r'/'
    dic = {}

    for iter in range(0,len(FileList)):
        cur_path = path + FileList[iter]
        dic[iter] = PreProcess(cur_path,iter)
        # print(dic[iter])
    return dic


def row_split(tag):
    FileList = FileReader(tag)
    volc = File_Dic(FileList,tag)
    j_data = json.dumps(volc,ensure_ascii=False)
    with open(r'./row_语料分词_' + tag + '.json' ,'w',encoding='utf-8') as f:
        f.write(j_data)

row_split('pos')
row_split('neg')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Windows\Temp\jieba.cache
Loading model cost 0.715 seconds.
Prefix dict has been built successfully.


In [3]:
# 把所有的句子load 进来
def loadSentence():
    sentence = []
    dic = {}
    path = '.\\row_语料分词_'
    with open(path + 'pos.json','r',encoding = 'utf-8') as f:
        dic = json.load(f)
        for iter in dic.values():
            for val in iter.values():
                sentence.append(val)
    with open(path + 'neg.json','r',encoding = 'utf-8') as f:
        dic = json.load(f)
        for iter in dic.values():
            for val in iter.values():
                sentence.append(val)
    return sentence
sentences = loadSentence()


In [4]:
# print(sentence[:10])

In [5]:
from gensim.models import word2vec

CBOW_Model = word2vec.Word2Vec(sentences, vector_size=64,sg = 0)
CBOW_Model.save("word2vec_CBOW.model")


skip_garm_Model = word2vec.Word2Vec(sentences, vector_size=64,sg = 0)
skip_garm_Model.save("word2vec_skip_garm.model")


In [6]:
# 训练好的 词向量
# https://radimrehurek.com/gensim/models/keyedvectors.html#
from gensim.models import KeyedVectors

# 二进制形式
# CBOW_vectors = model.wv
# CBOW_vectors.save('CBOWvectors.kv')
# CBOW_word_vectors = KeyedVectors.load('vectors.kv')

CBOW_Model.wv.save_word2vec_format('CBOW.txt',binary = False)
skip_garm_Model.wv.save_word2vec_format('skip_garm.txt',binary = False)


In [7]:
#读取模型
# model = word2vec.Word2Vec.load("your_model_name")
CBOW_Vector = CBOW_Model.wv
skip_garm_Vector = skip_garm_Model.wv
print(CBOW_Vector['干净'])

print(CBOW_Vector.most_similar('干净'))
print()
print(skip_garm_Vector.most_similar('差'))

[-0.34854105 -1.034342   -0.06974277  1.1922891  -0.5333772  -0.08679725
  0.9408814  -0.3173303  -0.13317023  0.18002288  1.2883686  -0.15165782
 -0.80739814 -0.28593966 -0.52885735  0.58958316 -0.22722445  0.22741541
 -0.880595    0.8154139   0.200517    1.1794848   0.6841546  -0.44262642
 -0.20810828  1.2707013  -1.1705346  -0.70590585 -0.10543224 -0.34392506
 -1.213578    0.5781314  -0.02455596 -1.3731282   0.96913016 -0.39055815
  0.41830233  0.1448541   0.73811644  0.33483657 -0.07545519  0.30129763
 -0.1441485  -0.5901611   0.02402375 -0.09789886 -0.62780035  0.01154778
  0.32810256  0.59840524  1.4017682  -0.4035343   2.2437356   0.32142228
 -0.05136845  0.82486135 -0.09535942 -0.14841291 -0.73174024  0.78472626
 -0.26722622 -0.77832407 -0.37597272  0.19153492]
[('旧', 0.9948569536209106), ('一般', 0.9941177368164062), ('陈旧', 0.9912216663360596), ('设施', 0.9896920323371887), ('比较', 0.9861384630203247), ('环境', 0.9759221076965332), ('但', 0.9756282567977905), ('隔音', 0.9750422835350037

In [8]:
# 利用已得到的词的向量表示做句子的向量表示
print(len(CBOW_Vector))
print(len(skip_garm_Vector))

3116
3116


In [9]:
print(sentences[0])
print('早餐' in CBOW_Vector)
print(CBOW_Vector['早餐'] * 2)

['早餐', '太', '差', '无论', '去', '多少', '人', '那边', '也', '不加', '食品', '的', '。', '酒店', '应该', '重视', '一下', '这个', '问题', '了', '。']
True
[ 0.23889136 -1.442047    0.27054727  1.5803475  -0.5371949  -0.68213487
  1.5501332  -0.38456663 -0.7432624  -0.33872926  1.7118111  -0.44120052
 -0.9703725  -0.25848624 -0.23643531  1.0129778  -0.4344748   0.2942489
 -1.5947871   1.7475039   0.5832866   1.9694809   0.88499707 -0.84228516
 -0.47149134  2.276378   -1.4875078  -0.8115608   0.20088314 -0.569311
 -1.428398    0.66019225 -0.6316423  -2.1347134   0.7661441  -0.89627963
  0.6127321   0.2956385   1.0628462   0.70144916 -0.01865558  0.64356625
 -0.3167452  -0.53170633  0.28982204 -0.30712095 -0.86900365 -0.11825174
  0.2330756   1.2551416   1.4775182  -0.6333071   3.0787601   0.83511007
  0.08379519  1.1118814   0.16943388 -0.876393   -1.3571746   1.2184899
 -0.29570964 -1.1671492  -1.0955456   0.3571478 ]


In [10]:
# 计算词的TF-idf

# 统计type and token
wordList = {}
for sentence in sentences:
    for word in sentence:
        if word in wordList:
            wordList[word] += 1
        else:
            wordList[word] = 1;


In [13]:
import math
import numpy as np
def softmax(x):
    x = np.array(x)
    # 计算每行的最大值
    row_max = np.max(x)
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    x = x - row_max
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp)
    s = x_exp / x_sum
    return s

In [14]:

VectorSentence = []
for sentence in sentences:
    tmp = np.zeros(64)
    for word in sentence:
        # wordcurFreq = sentence.count(word)
        # wordAllFreq = wordList[word]
        weight = float(sentence.count(word))/len(CBOW_Vector) * math.log(float(len(CBOW_Vector))/wordList[word])
        if word in CBOW_Vector:
            tmp += (CBOW_Vector[word] * weight)
        else:
            tmp += (np.zeros(64))
    
    VectorSentence.append(softmax(tmp))

In [15]:
np.save('句子的向量表示_bin',VectorSentence)

In [ ]:
# vec = np.load(r'./句子的向量表示_bin.npy')
# print (np.shape(vec))

(4000, 64)


In [16]:
# 存储
def GetLabel():
    dicPos = {}
    dicNeg = {}
    path = '.\\row_语料分词_'
    with open(path + 'pos.json','r',encoding = 'utf-8') as f:
        dicPos = json.load(f)
       
    with open(path + 'neg.json','r',encoding = 'utf-8') as f:
        dicNeg = json.load(f)
    return dicPos,dicNeg

dicPos,dicNeg = GetLabel()



In [17]:
print(len(dicPos))

2000


In [18]:
with open (r'./句子的向量表示.txt','w',encoding = 'utf-8') as f:
    cur = 0
    for i in range(0,len(dicPos)):
        f.write (str(i) + '\t' + str(list(VectorSentence[cur])) + '\n')
        cur += 1
    for iter in (VectorSentence[cur:]):
        f.write (str(i) + '\t' + str(list(iter)) + '\n')
        cur += 1